# Arnoldi iteration


## Krylov subspaces and the power iteration

An intuitive method for finding the largest (in absolute value) eigenvalue of a given m × m matrix $A$ is the power iteration: starting with an arbitrary initial vector b, calculate $Ab$, $A^2b$, $A^3b$,… normalizing the result after every application of the matrix $A$.


This sequence converges to the eigenvector corresponding to the eigenvalue with the largest absolute value, $\lambda _{1}$. However, much potentially useful computation is wasted by using only the final result, $A^{n-1}b$. This suggests that instead, we form the so-called Krylov matrix: $$ K_{n}=[ b\ Ab\ A^{2}b\ \cdots\ A^{n-1}b].$$


The columns of this matrix are not in general orthogonal, but we can extract an orthogonal basis, via a method such as Gram–Schmidt orthogonalization. The resulting set of vectors is thus an orthogonal basis of the Krylov subspace, $\mathcal{K}_{n}$. We may expect the vectors of this basis to span good approximations of the eigenvectors corresponding to the $n$ largest eigenvalues, for the same reason that $A^{{n-1}}b$ approximates the dominant eigenvector.

https://en.wikipedia.org/wiki/Arnoldi_iteration

## The Arnoldi iteration

The Arnoldi iteration uses the modified Gram–Schmidt process to produce a sequence of orthonormal vectors, $q_1, q_2, q_3, ...$, called the Arnoldi vectors, such that for every $n$, the vectors $q_1, q_2, q_3, ...$ span the Krylov subspace  ${\mathcal  {K}}_{n}$.

In [1]:
import numpy as np

def arnoldi_iteration(A, b, n: int):
    """Computes a basis of the (n + 1)-Krylov subspace of A: the space
    spanned by {b, Ab, ..., A^n b}.

    Arguments
      A: m × m array
      b: initial vector (length m)
      n: dimension of Krylov subspace, must be >= 1
    
    Returns
      Q: m x (n + 1) array, the columns are an orthonormal basis of the
        Krylov subspace.
      h: (n + 1) x n array, A on basis Q. It is upper Hessenberg.  
    """
    eps = 1e-12
    h = np.zeros((n+1,n))
    Q = np.zeros((A.shape[0],n+1))
     # Normalize the input vector
    Q[:,0] =b/np.linalg.norm(b,2)   # Use it as the first Krylov vector
    for k in range(1,n+1):
        v = np.dot(A,Q[:,k-1])  # Generate a new candidate vector
        for j in range(k):  # Subtract the projections on previous vectors
            h[j,k-1] = np.dot(Q[:,j].T, v)
            v = v - h[j,k-1] * Q[:,j]
        h[k,k-1] = np.linalg.norm(v,2)
        if h[k,k-1] > eps:  # Add the produced vector to the list, unless
            Q[:,k] = v/h[k,k-1]
        else:  # If that happens, stop iterating.
            return Q, h
    return Q, h

## Krylov subspace

Krylov subspaces are used in algorithms for finding approximate solutions to high-dimensional linear algebra problems. Many linear dynamical system tests in control theory, especially those related to controllability and observability, involve checking the rank of the Krylov subspace. These tests are equivalent to finding the span of the Grammians associated with the system/output maps so the uncontrollable and unobservable subspaces are simply the orthogonal complement to the Krylov subspace.

Modern iterative methods such as Arnoldi iteration can be used for finding one (or a few) eigenvalues of large sparse matrices or solving large systems of linear equations. They try to avoid matrix-matrix operations, but rather multiply vectors by the matrix and work with the resulting vectors. Starting with a vector $b$, one computes $Ab$, then one multiplies that vector by $A$ to find $A^{2}b$ and so on. All algorithms that work this way are referred to as Krylov subspace methods; they are among the most successful methods currently available in numerical linear algebra.


The Arnoldi procedure produces a basis $V_m$ of $\mathcal{K}_m$  and an upper Hesseberg matrix $H_m$ of dimension $m \times m$ with coefficients $h_{ij}$. We obtain $$ H_m = V_m^T A V_m .$$ Therefore $H_m$, is the projection of the linear transformation $A$ onto the Krylov subspace $\mathcal{K}_m$ with respect to the basis $V_m$.


Given that $H_m$ is a projection of the linear operator, an approximation can be made such that $$ e^{t A} v = |v| V_m e^{t H_m} e_1 .$$

http://indico.ictp.it/event/8344/session/55/contribution/226/material/slides/0.pdf